# Grasple Test to Microsoft Excel

Below you can upload a `.csv`-file exported from a Grasple test. After review of the generated output you can converted the output to a Microsoft Excel file available for download.

No data is stored on the server, as all Python code is run within your local browser. All data will be cleared after you close the browser tab or restart the Thebe kernel.

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
import base64
import micropip

# Widgets
upload_label = widgets.HTML('<b>Grasple CSV: </b>')
upload = widgets.FileUpload(accept='.csv', multiple=False)
upload_row = widgets.HBox([upload_label, upload])  # Label in front of button
output = widgets.Output()
download_button = widgets.Button(
    description="Generate Excel file",
    disabled=True,
    layout=widgets.Layout(width='auto')
)
output_link = widgets.Output()
download_row = widgets.HBox([download_button, output_link])

# Install xlsxwriter using micropip
await micropip.install("xlsxwriter")

def create_download_link(df):
    """Create a downloadable link for the DataFrame as an Excel file with formatted columns."""
    with io.BytesIO() as buffer:
        with pd.ExcelWriter(buffer, engine="xlsxwriter") as writer:
            worksheet = writer.book.add_worksheet("Sheet1")
            # Define formats
            right_align_format = writer.book.add_format({'align': 'right'})
            right_align_format.set_border(1)
            right_align_format.set_bg_color('#CCF1F4')
            light_right_align_format = writer.book.add_format({'align': 'right'})
            light_right_align_format.set_border(1)
            light_right_align_format.set_bg_color('#E6F8FA')
            bold_right_align_format = writer.book.add_format({'align': 'right',"bold": True})
            bold_right_align_format.set_border(1)
            bold_right_align_format.set_bottom(5)
            bold_right_align_format.set_bg_color('#00B8C8')
            left_align_format = writer.book.add_format({'align': 'left'})
            left_align_format.set_border(1)
            left_align_format.set_bg_color('#CCF1F4')
            light_left_align_format = writer.book.add_format({'align': 'left'})
            light_left_align_format.set_border(1)
            light_left_align_format.set_bg_color('#E6F8FA')
            bold_left_align_format = writer.book.add_format({'align': 'left',"bold": True})
            bold_left_align_format.set_border(1)
            bold_left_align_format.set_bottom(5)
            bold_left_align_format.set_bg_color('#00B8C8')
            center_align_format = writer.book.add_format({'align': 'center'})
            center_align_format.set_border(1)
            center_align_format.set_bg_color('#CCF1F4')
            light_center_align_format = writer.book.add_format({'align': 'center'})
            light_center_align_format.set_border(1)
            light_center_align_format.set_bg_color('#E6F8FA')
            bold_center_align_format = writer.book.add_format({'align': 'center',"bold": True})
            bold_center_align_format.set_border(1)
            bold_center_align_format.set_bottom(5)
            bold_center_align_format.set_bg_color('#00B8C8')

            worksheet.write(0, 0, "Student number", bold_right_align_format)  # Write index with right alignment
            worksheet.write(0, 1, "Student name", bold_left_align_format)  # Write index with right alignment
            for i, val in enumerate(df.index.values):
                if i % 2 == 0:
                    worksheet.write(i+1, 0, val[0], right_align_format)  # Write index with right alignment
                    worksheet.write(i+1, 1, val[1], left_align_format)
                else:
                    worksheet.write(i+1, 0, val[0], light_right_align_format)
                    worksheet.write(i+1, 1, val[1], light_left_align_format)
            for col,name in enumerate(df.columns.values):
                worksheet.write(0, col+2, name, bold_center_align_format)
                for row, val in enumerate(df[name].values):
                    if row % 2 == 0:
                        worksheet.write(row+1, col+2, val, center_align_format)
                    else:
                        worksheet.write(row+1, col+2, val, light_center_align_format)

                
            # df.to_excel(writer, index=True, sheet_name="Sheet1")

            worksheet.autofit()
        buffer.seek(0)
        b64 = base64.b64encode(buffer.read()).decode()
        href = f'data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}'
        return href

def on_download_click(b):
    """Handle the download button click."""
    download_button.disabled = True  # Disable after click
    output_link.clear_output()
    with output_link:
        try:
            # Create the download link
            download_link = create_download_link(pvt)
            # Display the download link
            display(widgets.HTML(
                f'<a download="output.xlsx" href="{download_link}" target="_blank">Download Excel</a>'
            ))
        except Exception as e:
            print("❌ Error creating download link:")
            print(e)

def on_upload_change(change):
    output.clear_output()
    global pvt  # Declare pvt as global to use it in the download button
    if upload.value:
        with output:
            try:
                # Retrieve the first file from the tuple
                uploaded_file = upload.value[0] if isinstance(upload.value, tuple) else list(upload.value.values())[0]
                if isinstance(uploaded_file, dict):
                    content = uploaded_file['content']
                else:
                    content = uploaded_file[1]['content']
                df = pd.read_csv(io.BytesIO(content))
                df = df.drop(df[df['started_test'] == 0].index)
                df['main_exercise_id'] = df['main_exercise_id'].astype(int)
                print("✅ CSV successfully loaded.")
                # Convert to pivot table
                question_pool_names_temp = df['question_pool_name'].unique()
                question_pool_names = [
                    name for name in question_pool_names_temp if isinstance(name, str)
                ]
                main_exercise_ids_temp = df['main_exercise_id'].unique()
                main_exercise_ids = [
                    number for number in main_exercise_ids_temp if isinstance(number, str)
                ]
                if question_pool_names:
                    pvt = pd.pivot_table(
                        df, 
                        values="scored_points", 
                        index=["student_id", "user_name"], 
                        aggfunc="sum", 
                        fill_value=0, 
                        columns="question_pool_name"
                    )
                    times = pd.pivot_table(
                        df, 
                        values="exercise_received_at", 
                        index="student_id", 
                        aggfunc="min", 
                        fill_value="missing", 
                        columns="question_pool_name"
                    )
                    for name in question_pool_names:
                        times = times.drop(times[times[name] == "missing"].index)
                    times = times.iloc[:1].T
                    times.rename(columns={times.columns[0]: 'times'}, inplace=True)
                    times.sort_values(by='times', inplace=True)
                    pvt = pvt[times.index]
                else:
                    pvt = pd.pivot_table(
                        df, 
                        values="scored_points", 
                        index=["student_id", "user_name"], 
                        aggfunc="sum", 
                        fill_value=0, 
                        columns="main_exercise_id"
                    )
                    times = pd.pivot_table(
                        df, 
                        values="exercise_received_at", 
                        index="student_id", 
                        aggfunc="min", 
                        fill_value="missing", 
                        columns="main_exercise_id"
                    )
                    for name in main_exercise_ids:
                        times = times.drop(times[times[name] == "missing"].index)
                    times = times.iloc[:1].T
                    times.rename(columns={times.columns[0]: 'times'}, inplace=True)
                    times.sort_values(by='times', inplace=True)
                    pvt = pvt[times.index]
                display(pvt)
                download_button.disabled = False  # Enable the download button
                download_button.layout.display = 'block'  # Show the download button
            except Exception as e:
                print("❌ Error loading CSV:")
                print(e)

# Hide the download button initially
download_button.layout.display = 'none'

upload.observe(on_upload_change, names='value')
download_button.on_click(on_download_click)

# Display the widgets
display(upload_row, output, download_row)


Output()